<a href="https://colab.research.google.com/github/seungmei/ESAA_23_2/blob/main/%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B83_ridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 및 라이브러리 로딩

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/movies/movies_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/movies/movies_test.csv')

## 데이터탐색

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [ ]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [ ]:
test.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7
3,의형제,(주)쇼박스,액션,2010-02-04,116,15세 관람가,장훈,6.913420e+05,2,408,2
4,평행 이론,CJ 엔터테인먼트,공포,2010-02-18,110,15세 관람가,권호영,3.173800e+04,1,380,1


#####결측치 처리

In [ ]:
# 결측치
train.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64

In [ ]:
# 결측치가 많은 데이터를 평균으로 대체
import numpy as np

print(np.mean(train["dir_prev_bfnum"]))
print(np.mean(test["dir_prev_bfnum"]))

train["dir_prev_bfnum"].fillna(1050443,inplace = True)
test["dir_prev_bfnum"].fillna(891670,inplace = True)

1050442.8914197532
891669.5160436138


##### 피처 제거

In [ ]:
# 감독명 : 너무 다양해서 제거
train = train.drop(['director'],axis = 1)
test = test.drop(['director'],axis = 1)

In [ ]:
# 제목 : 의미가 없기 때문에 제거
train = train.drop(['title'],axis= 1)
test = test.drop(['title'],axis= 1)

##### 배급사 전처리

In [ ]:
train.distributor.value_counts()

CJ 엔터테인먼트        54
롯데엔터테인먼트         52
(주)NEW           30
(주)마운틴픽쳐스        29
(주)쇼박스           26
                 ..
OAL(올)            1
(주)에이원 엔터테인먼트     1
(주)콘텐츠 윙          1
위더스필름             1
퍼스트런              1
Name: distributor, Length: 169, dtype: int64

##### 배급사별 영화 관객수 중위값 알아보기

In [ ]:
dis_rank = train.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')
dis_rank

,distributor,num_rank
141,인피니티엔터테인먼트,2.0
65,고구마공작소,8.0
92,사람과 사람들,42.0
131,위드시네마,46.0
68,나우콘텐츠,54.0
...,...,...
50,CJ E&M 영화부문,2242510.0
121,영구아트무비,2541603.0
96,쇼박스(주)미디어플렉스,2634823.0
27,(주)아이필름코퍼레이션,3117859.0


In [ ]:
dis_rank['num_rank'] = [i + 1 for i in range(dis_rank.shape[0])]
dis_rank

,distributor,num_rank
141,인피니티엔터테인먼트,1
65,고구마공작소,2
92,사람과 사람들,3
131,위드시네마,4
68,나우콘텐츠,5
...,...,...
50,CJ E&M 영화부문,165
121,영구아트무비,166
96,쇼박스(주)미디어플렉스,167
27,(주)아이필름코퍼레이션,168


##### 개봉일을 바탕으로 년,월 변수 생성

In [ ]:
train['년'] = train['release_time'].apply(lambda x: int(x[:4]))
train['월'] = train['release_time'].apply(lambda x: int(x[5:7]))
train =  train.drop(['release_time'],axis = 1)

test['년'] = test['release_time'].apply(lambda x: int(x[:4]))
test['월'] = test['release_time'].apply(lambda x: int(x[5:7]))
test =  test.drop(['release_time'],axis = 1)

## 모델정의 및 학습

#### log 변환

In [ ]:
train = pd.merge(train, dis_rank, how="left")
test = pd.merge(test, dis_rank, how='left')
test.fillna(0, inplace = True)
train.head(3)

,distributor,genre,time,screening_rat,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,년,월,num_rank
0,롯데엔터테인먼트,액션,96,청소년 관람불가,1050443.00,0,91,2,23398,2012,11,151
1,(주)쇼박스,느와르,130,청소년 관람불가,1161602.50,2,387,3,7072501,2015,11,164
2,(주)쇼박스,액션,123,15세 관람가,220775.25,4,343,4,6959083,2013,6,164


In [ ]:
# 로그값으로 변환하기
X = train[['num_rank', 'time', 'num_staff', 'num_actor', 'screening_rat', 'dir_prev_bfnum', 'genre']]
y = np.log1p(train.box_off_num)
X = pd.get_dummies(columns=['screening_rat'], data=X)
X = pd.get_dummies(columns=['genre'], data=X)
X['dir_prev_bfnum'] = np.log1p(X['dir_prev_bfnum'])
X['num_actor'] = np.log1p(X['num_actor'])
X['num_staff'] = np.log1p(X['num_staff'])

test_log = test[['num_rank', 'time', 'num_staff', 'num_actor', 'screening_rat', 'dir_prev_bfnum', 'genre']]
test_log = pd.get_dummies(columns=['screening_rat'], data=test_log)
test_log = pd.get_dummies(columns=['genre'], data=test_log)
test_log['dir_prev_bfnum'] = np.log1p(test_log['dir_prev_bfnum'])
test_log['num_actor'] = np.log1p(test_log['num_actor'])
test_log['num_staff'] = np.log1p(test_log['num_staff'])

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

# 릿지에 사용될 alpha 파라미터의 값을 정의
alphas = [0, 0.001, 0.01, 0.1 , 1, 10, 100]
# alpha를 낮게 = 규제 약하게 = 데이터에 더 많은 피팅을 하여라 => 과대적합에 가깝게
# alpha를 높게 = 규제 강하게 = 모든 계수 값을 0에 가깝게 => 과소적합에 가깝게

# alpha list 값을 반복하면서 alpha에 따른 평균 RMSE를 구함.
for alpha in alphas:
  ridge_log = Ridge(alpha = alpha)
  neg_mse_scores = cross_val_score(ridge_log, X, y, scoring='neg_mean_squared_error',cv=5)
  avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
  print('alpha {0} 일 때 5 folds의 평균 RMSE : {1:.3f} '.format(alpha, avg_rmse))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.46194e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


alpha 0 일 때 5 folds의 평균 RMSE : 1.700 
alpha 0.001 일 때 5 folds의 평균 RMSE : 1.651 
alpha 0.01 일 때 5 folds의 평균 RMSE : 1.651 
alpha 0.1 일 때 5 folds의 평균 RMSE : 1.650 
alpha 1 일 때 5 folds의 평균 RMSE : 1.647 
alpha 10 일 때 5 folds의 평균 RMSE : 1.635 
alpha 100 일 때 5 folds의 평균 RMSE : 1.630 


In [ ]:
ridge_log= Ridge(alpha=100)
ridge_log.fit(X, y)
pred_ridge_log = ridge_log.predict(test_log)

# 학습 된 모델로 예측 데이터 생성

## 제출파일 생성

In [ ]:
submission_log = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/movies/submission.csv')
submission_log

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0
...,...,...
238,해에게서 소년에게,0
239,울보 권투부,0
240,어떤살인,0
241,말하지 못한 비밀,0


In [ ]:
submission_log['box_off_num'] = pred_ridge_log

In [ ]:
submission_log

,title,box_off_num
0,용서는 없다,13.073257
1,아빠가 여자를 좋아해,13.597171
2,하모니,13.331210
3,의형제,13.819413
4,평행 이론,13.311805
...,...,...
238,해에게서 소년에게,3.858848
239,울보 권투부,8.195530
240,어떤살인,5.957599
241,말하지 못한 비밀,5.067632


In [ ]:
submission_log.to_csv('/content/drive/MyDrive/Colab Notebooks/dataset/movies/submission_ridge_log.csv',index = False)